Для решения мы использовали не машинно обученную модель.
1. Отсекали шоу по доле просмотра пользователем с коэф. 0.5  
2. Далее для каждого пользователя брали все его просмотры за неделю и пытались сагрегировать аналогичные просмотры в будущей программе.
3. Выбирали максимальные по количеству просмотров программы, которые пользователь просмотрел бы в будущем, если бы смотрел в том же режиме.
4. Не хватающие позиции у пользователей добивались самыми популярными среди всех.
5. Фильтрация шоу по тем, которые уже смотрел пользователь также увеличила скор.

Private 0.42544 / 4 место

Public 0.43511 / 4 место

In [1]:
import pandas as pd
from pandas import Series, DataFrame

import numpy as np

import sys
sys.path.append('../')

import tqdm
import joblib
from movies_utils.nnet_sim import *
from movies_utils.tv_utils import *

In [2]:
%%time
data = pd.read_csv('../channels/TV_program_AV/merged_log_with_show_ids_1119_new.csv')
mapping = joblib.load('../channels/TV_program_AV/number_to_user_id.pickle')
data['user_id'] = data['user_id'].map(mapping).values

for col in ['session_start_time', 'session_stop_time', 'show_start_time', 'show_end_time']:
    data[col] = data[col].astype(np.datetime64)
    
data['s_start_time'], data['s_end_time'] = create_wdts(data['session_start_time'], 
                                                       data['session_stop_time'])

data['watch_duration'] = data[['session_stop_time', 'show_end_time']].min(axis=1) \
        - data[['session_start_time', 'show_start_time']].max(axis=1)

data['watch_duration'] /= np.timedelta64(1, 's')
data['watch_rate'] = data['watch_duration'] / data['show_duration'] 

CPU times: user 28.3 s, sys: 3.59 s, total: 31.8 s
Wall time: 31.8 s


In [3]:
prgs = get_programs('../channels/new_raw_data/export_arh_11-20-final.csv',
'../channels/new_raw_data/export_arh_21-30-final.csv',
'../channels/new_raw_data/export_arh_31-42-final.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
baseline = data[data['watch_rate'] >= .5]
baseline = baseline['tv_show_id'].value_counts()[:5].index.tolist()

In [5]:
baseline

[2400480, 240081, 12001682, 20088, 2400508]

### VALIDATION PART

In [6]:
train_co = prgs['valid']['start_time'].min()

train = data[data['show_start_time'] <= train_co]
valid = data[data['show_start_time'] > train_co]
valid = valid[valid['user_id'].isin(set(train['user_id']))]

In [7]:
valid_target = get_target(valid, watch_rate_co = .8, top_k = 5, add_empty=False)

In [8]:
valid_pred = get_target(train, watch_rate_co = .8, top_k = 5, add_empty=True).loc[valid_target.index]

In [9]:
map5(valid_target, get_pred_with_baseline(valid_pred, baseline))

0.25450243866928735

In [10]:
tr_short = train# [train.groupby('user_id')['session_start_time'].transform(lambda x: cut_hist(x, 45))]

tr_short = tr_short[['user_id', 'channel_id', 's_start_time', 's_end_time']].drop_duplicates()
prg_short = prgs['valid'][['channel_id', 'w_start_time', 'w_end_time', 'tv_show_id']]

In [11]:
merged = merge_batch(tr_short, prg_short, batch_size = 3)

In [12]:
merged = process_merged(merged)

In [16]:
def filter_merged(merged, vals = None, train = None, score = None):
    if vals is None:
        vals = []
    merged = merged[merged['tv_show_id'].isin(vals)]
    merged = merged[merged['rate'] > .5]
    
    train_scores = None
    

    
    return merged

In [17]:
merged_upd = filter_merged(merged, vals = set(train['tv_show_id']), train=train, score=score)

In [18]:
merged_upd.shape

(2672034, 11)

In [19]:
score

20088        0.911765
30066        1.416667
30085        0.505882
52641        1.000000
90054        1.000000
               ...   
120082458    1.000000
120082460    1.000000
120082513    1.000000
120082514    1.000000
120082714    1.000000
Name: tv_show_id, Length: 7361, dtype: float64

In [22]:
new_pred = get_merged_pred(merged_upd, valid_pred.index)

In [24]:

def get_merged_pred(merged, ids):
    
    merged = merged.copy()

    merged = merged.sort_values('time', ascending=False)

    grp = merged.groupby('user_id')['tv_show_id'].agg(lambda x: x.value_counts()[:5].index.tolist())
    new_pred = grp.loc[[x for x in ids if x in grp.index]]
    new_ids = list(set(ids) - set(new_pred.index))
    add = Series([[]] * len(new_ids), index=new_ids)
    new_pred = pd.concat([new_pred, add], axis=0).loc[ids]

    return new_pred

In [25]:
map5(valid_target, get_pred_with_baseline(new_pred, valid_pred, baseline))

0.2720026206595326

In [26]:
map5(valid_target, [baseline] * len(valid_target))

0.23077091067918756

In [27]:
blended = blend_predictions(new_pred, valid_pred)

In [28]:
map5(valid_target, get_pred_with_baseline(blended, baseline))

0.26536361651015505

### TEST PART

In [48]:
ss = pd.read_csv('../channels/new_raw_data/submission.csv')

In [49]:
# valid prediction
test_pred = get_target(data, add_empty=True, add_ids = [10246214960655461086, 16561743585370579180])

In [50]:
test_pred = test_pred.loc[ss['user_id']]
test_pred = get_pred_with_baseline(test_pred, baseline)
ss['tv_show_id'] = [' '.join(map(str, x)) for x in test_pred]

In [51]:
tr_short = data[['user_id', 'channel_id', 's_start_time', 's_end_time']].drop_duplicates()
prg_short = prgs['test'][['channel_id', 'w_start_time', 'w_end_time', 'tv_show_id']]

In [52]:
merged = merge_batch(tr_short, prg_short, batch_size = 3)

In [53]:
merged = process_merged(merged)

In [54]:
merged_upd = filter_merged(merged, vals = set(data['tv_show_id']), train = data)

In [55]:
new_pred = get_merged_pred(merged_upd, ss['user_id'])

In [56]:
ss['tv_show_id'] = [' '.join(map(str, x)) for x in new_pred]

In [57]:
ss.isnull().sum()

user_id       0
tv_show_id    0
dtype: int64

In [58]:
ss.to_csv('with_empty_res_v0.csv', index=False)

In [59]:
test_pred = get_pred_with_baseline(new_pred, baseline)

In [60]:
ss['tv_show_id'] = [' '.join(map(str, x)) for x in test_pred]

In [61]:
ss.to_csv('tv_prg_pred_v3.csv', index=False)